<a href="https://colab.research.google.com/github/Aananda-giri/scripts/blob/main/zoom_video_download_v3_with_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')


!apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
!npm i convert-excel-to-json
!npm install puppeteer
!npm install puppeteer-extra
!npm install puppeteer-extra --save
!npm install puppeteer-extra-plugin-stealth --save   # installs plugin
!npm install puppeteer-extra-plugin-recaptcha

## Install google chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb   # Install the package using the dpkg command:
!sudo dpkg -i google-chrome-stable_current_amd64.deb       # If you encounter any dependency errors during the installation, you can resolve them by running the following command:
!sudo apt-get install -f      # Verify the installation by running the following command:
!google-chrome --version

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,080 B]
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http:/

## Search by Subject

Save links to json file because I coudn't  split date which is :link['E'], was converted to UTC and was showing err. link.E.getFullYear is not a function.

In [40]:
%%writefile ./filter_by_search.js
// check file that does not exist from links
// may miss two classes same day

var fs = require('fs');
const excelToJson = require('convert-excel-to-json');
const path = require('path');
const download_root = './drive/MyDrive/zoom_downloads/'

// function to save_json_data
let save_json_data = function (data, file_path){
  let to_save_data  = JSON.stringify(data);
try {
    fs.writeFileSync(file_path, to_save_data);
} catch (error) {
    console.log('Error saving data to file error_data...' + file_path + error);
}
}
// function to load json data :: to test if data is actually saved
let load_json_data = (file_path) => {
  // loading error links
  // var saved_data;
  try {
      var saved_data = fs.readFileSync(file_path, 'utf-8');
      saved_data = JSON.parse(saved_data);
      // console.log('Loaded Links...: \n ' + saved_data);
      return saved_data;
  } catch (error) {
      console.log('Error Loading json file ...: \n ' + file_path + error); 
  }
}

// function to search
let filter = function (sheet_links, search_terms){
      let search_results = [];
      let matchFound = false;
      
      for (let [current_link_index, link] of sheet_links.entries() ){
      // console.log(current_link_index, link)
      if (link['A'] == 'Class Name'){
        continue;
      }
      // console.log(link['A']);
      matchFound = false;
      for (let j = 0; j < search_terms.length; j++) {
        //console.log(`A: ${typeof(link['A'])} :: ${link['A']} ${link['A'].toString().includes(search_terms[j])}  ${search_terms[j]}\n `)
        //console.log(`B: ${typeof(link['B'])} :: ${link['B']} \n `)
        //console.log(`G: ${typeof(link['G'])} :: ${link['G']} \n `)
        // if (( (link['A'] != undefined) && (link['A'].toString().toLowerCase().includes(search_terms[j])) || (link['B'].toString().toLowerCase().includes(search_terms[j])) || (link['G'].toString().toLowerCase().includes(search_terms[j]) )))  {
        if ( (link['B'].toString().toLowerCase().includes(search_terms[j])) )  {
            matchFound = true;
            // console.log('match found')
            break;
        }
      }
      if (matchFound==true) {
        search_results.push(link);
      }
    }
    return search_results;
}




var all_sheets_links = excelToJson({  sourceFile: download_root + 'Fuse-Links.xlsx'  });
// console.log(all_sheets_links)

let subjects = ['physics', 'math', 'electronics', 'circuit','machine', 'economics', 'Electromagnetism', 'control system', 'intelligence', 'digital signal analysis', 'elective', 'principles of management', 'telecommunication']
let teachers = ['PUSHKAR RAJ POKHREL', '']
let results = [];

for (let current_sheet in all_sheets_links){
    // console.log(current_sheet)
    let sheet_links = all_sheets_links[current_sheet]
    console.log(sheet_links.length)
    results = [...results, ...filter(sheet_links, subjects)]

}










var links_to_download = []

save_json_data(results, download_root + 'links.json')


all_links_saved = load_json_data(download_root + 'links.json')
console.log('all_links_saved', all_links_saved.length)
if (all_links_saved) {
  console.log('---------------------------------------')
  console.log('successfully saved all links')
  console.log('---------------------------------------')
  }

Writing ./filter_by_search.js


In [ ]:
!node filter_by_search.js

7288
all_links_saved 1669
---------------------------------------
successfully saved all links
---------------------------------------


(python code)<br>
saving link_indexes yet to be download at file: ./drive/MyDrive/zoom_downloads/progress_logs_v3.json

In [41]:
%%writefile ./python_code.py
import json
import os
download_root = './drive/MyDrive/zoom_downloads/'
os.path.exists(download_root)


toDownload = []
toDownloadData = []
# data = []
with open(download_root + 'links.json','r') as file:
  links = json.load(file)
links = links[:]
for index,link in enumerate(links[1:]):
      if index == 1: continue  # pass first row (title row)
      
      link['B'] = link['B'].replace('/','|')
      link['C'] = link['C'].replace('/','|')
      link['G'] = link['G'].replace('/','|')
      corresponding_folder_name = link['B'] + '[' + link['C'] + '][' + link['G'] + ']'

      # file doesnot exists so need to downloads
      if not os.path.exists(download_root + corresponding_folder_name):
        # print(f'folder doesn\'t exist {corresponding_folder_name}')
        toDownload.append(index)
        toDownloadData.append({'url': link['J'], 'password':link['I'], 'path':corresponding_folder_name})
        continue
      
      
      
      year = link['E'].split('-')[0]
      month = link['E'].split('-')[1]
      day = link['E'].split('-')[2][:2]
      
      # print(year, month, day)
      
      corresponding_file_name_begin = 'GMT' + year + month + day + '-'
      corresponding_file_name_end = '.mp4'

      file_exists = False
      
      actual_files = os.listdir(download_root + corresponding_folder_name)
      for actual_file_name in actual_files:
        if (actual_file_name.startswith(corresponding_file_name_begin) and actual_file_name.endswith(corresponding_file_name_end)):
      
              file_exists = True
              break
          
      if (not file_exists):
          toDownload.append(index)
          toDownloadData.append({'url': link['J'], 'password':link['I'], 'path':corresponding_folder_name})
      # data.append({'folder':corresponding_folder_name, 'file_name_begin': corresponding_file_name_begin})
print('index',index)
print('todownload', len(toDownload))
print(toDownload)

with open(download_root + 'progress_logs_v3.json','w') as file:
  json.dump({'borrowed':[], 'to_download':toDownload}, file)

with open(download_root + 'to_download_data_v3.json','w') as file:
  json.dump({'data':toDownloadData}, file)

print('---------------------------------------')
print('successfully saved to_download indexes at progress_logs_v3.json')
print('---------------------------------------')

Writing ./python_code.py


In [ ]:
!python3 python_code.py

V3.1 : Use to_download_data_v3.json instead of progress_logs_v3.json

In [42]:
# actual code to download
# stores code to file: 'app_v3.js'
%%writefile ./drive/MyDrive/zoom_downloads/download_code_v3_1.js
'use strict';

// for colab code
const linksPath = './drive/MyDrive/zoom_downloads/Fuse-Links.xlsx';
const download_root = './drive/MyDrive/zoom_downloads/';  // create a folder called  'zoom_downloads' in MyDrive where downloads are to be stored

// for running code locally
// var download_root = '/home/gayatri/Documents/college/zoom_downloads/downloads/';
// var linksPath = '/home/gayatri/Documents/college/zoom_downloads/Fuse-Links.xlsx';  // colab: './drive/MyDrive/zoom_downloads/Fuse-Links.xlsx'


const excelToJson = require('convert-excel-to-json');
// const puppeteer = require('puppeteer');
const puppeteer = require('puppeteer-extra')
// add stealth plugin and use defaults (all evasion techniques)
const StealthPlugin = require('puppeteer-extra-plugin-stealth')
puppeteer.use(StealthPlugin())

var fs = require('fs'); // to create folder if not exist // reference: https://colab.research.google.com/drive/168X6Zo0Yk2fzEJ7WDfY9Q_0UOEmHSrZc?usp=sharing
const { ConsoleMessage } = require('puppeteer')
var progress_stored_previoiusly = 0;

// add recaptcha plugin and provide it your 2captcha token (= their apiKey)
// 2captcha is the builtin solution provider but others would work as well.
// Please note: You need to add funds to your 2captcha account for this to work
const RecaptchaPlugin = require('puppeteer-extra-plugin-recaptcha')
puppeteer.use(
  RecaptchaPlugin({
    provider: {
      id: '2captcha',
      token: '' // REPLACE THIS WITH YOUR OWN 2CAPTCHA API KEY ⚡
    },
    visualFeedback: true // colorize reCAPTCHAs (violet = detected, green = solved)
  })
)


const delay = ms => new Promise(resolve => {
    console.log("sleeping for " + ms/1000 + " s");
    setTimeout(resolve, ms);
});

// syncronous delay
function delay_sync(ms) {
  console.log("syncronous delay  " + ms/1000 + " s");
  return new Promise(resolve => setTimeout(resolve, ms));
}

// const randomInteger = require('random-int');
const randomInteger = (min, max) => {
  return Math.floor(Math.random() * (max - min + 1)) + min;
}



// create 'progress_logs_v3.json' if file doesnot exist
if (!fs.existsSync(download_root + 'progress_logs_v3.json')){
  console.log('\ncreating file: progress_logs_v3.json...\n');
  fs.writeFileSync(download_root + 'progress_logs_v3.json', JSON.stringify(
    {
      "comment":"progress for sheets are generated and updated based on sheet_name automatically .."
    }
  ));
 }

// Create 'error_links_logs' if file doesnot exist
if (!fs.existsSync(download_root + 'error_logs.json')){
  console.log('\ncreating file: error_logs.json...\n');
  fs.writeFileSync(download_root + 'error_logs.json', JSON.stringify(
      {
          link_logs:[],
          other_logs:[],
      }
      ));
}

let load_json_data = (file_path) => {
  // loading error links
  // var saved_data;
  try {
      var saved_data = fs.readFileSync(file_path, 'utf-8');
      saved_data = JSON.parse(saved_data);
      // console.log('Loaded Links...: \n ' + saved_data);
      return saved_data;
  } catch (error) {
      console.log('Error Loading json file ...: \n ' + file_path + error); 
  }
}


let save_json_data = function (data, file_path){
  let to_save_data  = JSON.stringify(data);
try {
    fs.writeFileSync(file_path, to_save_data);
} catch (error) {
    console.log('Error saving data to file error_data...' + file_path + error);
}
}

// load progress logs
// data = {'borrowed':[0,1]}
var data = load_json_data(download_root + 'progress_logs_v3.json');
var borrowed = [...data.borrowed]       // links download in progress
console.log('\nborrowed: ',borrowed)

// update current progress of sheet after asynchronously waiting for download_time seconds. 
let async_wait_and_update_current_download_progress =  async (how_long_after_to_assume_downloaded, borrowed) => {
  setTimeout(function(){
    save_json_data({'borrowed':[...borrowed]}, download_root + 'progress_logs_v3.json');
    console.log('updated borrowed: ', borrowed)
      // let current_date_ms = Date.parse(new Date());
      // console.log(`updated download index to: ${current_link_index} for sheet: ${current_sheet}`);

  }, how_long_after_to_assume_downloaded);//wait 2 seconds
}






let append_error_logs = (new_log, where) => {
  // where = 'link_logs' or 'other_logs'

  let previous_logs = load_json_data(download_root + 'error_logs.json');  // load previous logs
  // console.log('previous_logs' + previous_logs);
  // console.log('prev' + previous_logs);
  previous_logs[where].push(new_log);                   // update logs
  // console.log(previous_logs[where]);
  
  let initial_date = Date.now()
  
  while (Date.now() <= initial_date + 10000){}
  save_json_data(previous_logs, download_root + 'error_logs.json');    // store error logs
  previous_logs = JSON.stringify(previous_logs);
  console.log('\nappended error log ...\n');
  console.log('\n waiting 15 minutes sync for remaining downloads to finis \n')
  // update links.indexOf
  delay_sync(900000); // waiting 15 minutes for remaining downloads to finish
}


// js progress generator function
function progress_bar(current_progress, total, label){
  let progress = Math.round((current_progress/total)*100);
  console.log(`${label}_progress  : ${progress}% :: `, current_progress, '/', total);
  console.log(total);
  let bar = [];
  for (let i = 0; i < 100; i++){
    if (i < progress){
      bar.push('█');
    } else {
      bar.push('░');
    }
  }
  // return bar.join(''); // array to string
  console.log(bar.join(''));
}
// for (let i=0;i<10000;i++ ){progress_bar(i,10000, 'count')} # test progress_bar


function exist_in_error_logs(url){
    let error_logs = load_json_data(download_root + 'error_logs.json');
    for (let error_url of error_logs['link_logs'] ){
      // console.log('linke: ',error_url);
      // console.log('\nurle:',error_url.url);
      if (error_url['url'] == url){
        return true;
      }
    }
    return false;
    // console.log(error_logs['link_logs'].length, error_logs['link_logs'])
}



const download_links = async (links, skip_error_logs=true) => {
    // initialize browser
    // refrence: https://colab.research.google.com/drive/168X6Zo0Yk2fzEJ7WDfY9Q_0UOEmHSrZc?usp=sharing
    // google cloud console
    // const browser = await puppeteer.launch({executablePath:"/opt/google/chrome/google-chrome", args:['--no-sandbox','--start-maximized'], ignoreHTTPSErrors: true, headless: true});
    // colab specific
    // const browser = await puppeteer.launch({executablePath:"/usr/bin/chromium-browser", args:['--no-sandbox','--start-maximized'], ignoreHTTPSErrors: true, headless: true});
    // const browser = await puppeteer.launch({headless:false, ignoreHTTPSErrors: true}); // colab: const browser = await puppeteer.launch({executablePath:"/usr/bin/chromium-browser", args:['--no-sandbox','--start-maximized'], ignoreHTTPSErrors: true, headless: true});  // colab code:  
    const browser = await puppeteer.launch({executablePath: "/opt/google/chrome/chrome", args:['--no-sandbox','--start-maximized'], ignoreHTTPSErrors: true, headless: true});
  
    const page = await browser.newPage();
    await page.setExtraHTTPHeaders({
        'Accept-Language': 'en-US,en;q=0.9',
    });
    await page.setUserAgent('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36');
    
    // accept cookies
    // console.log('opening https://zoom.us/ and accepting cookies ...');
    // await page.goto('https://zoom.us/', {timeout: 60000});//, {waitUntil: 'networkidle0'});
    // await page.screenshot({path: screenShotPath + 'before-accept-cookies' +'.png'});
    // await page.waitForSelector('#onetrust-accept-btn-handler');
    // document.querySelector('#onetrust-accept-btn-handler').click()
    // await page.screenshot({path: screenShotPath + 'after-accept-cookies' +'.png'});
    // console.log('accepted cookies ...')

    // await delay(2000);  // wait 2 seconds

    // file_name
    // f'[count_index] + [{class_name[:170]}] + [{course_name}] + [{Grade}] + [{Duration}] + [{StartDateTime}] + [{EndDateTime}] + [{InstructorName}]

      const screenShotPath = download_root + "screenshots/";
      // for (let [current_link_index, link] of links.entries() ){
      //for(let current_link_index of to_download){
      for (let link of links){
        
        console.log('link', link['url'], link['path']);
        
        if (skip_error_logs==true){
          // Skip url if it error occored in link previously
          if (exist_in_error_logs(link['url'])){
            console.log('\n skipping! url in error_logs');
            continue;
          }
        }
        
        try {
            let url = link['url']
            let path = link['path']
            let password = link['password']
            let current_link_index = links.indexOf(link)
            if (borrowed.indexOf(current_link_index) != -1){
              // skip if borrowed
              console.log(`\n link: ${current_link_index} in borrowed skipping...\n`);
              continue;
            } else {
              // add link to borrowed list
              borrowed.push(current_link_index);
              save_json_data({'borrowed':[...borrowed]}, download_root + 'progress_logs_v3.json');
            }
            progress_bar(current_link_index, links.length, 'download');  // displays progress of download

          if (url.slice(0,4) !='http') {
            append_error_logs({courseName: path, password: password, url: url, error_value: 'url not link'},'other_logs');
            continue; // skip non download url
        }
        const downloadPath = download_root + path;
        console.log('point1');
        // create folder if not exists
        if (!fs.existsSync(downloadPath)){
            fs.mkdirSync(downloadPath);
            console.log(`creating folder: ${downloadPath}`);
            
            if (!fs.existsSync(screenShotPath)){    // create screenshot path if don't exist
                fs.mkdirSync(screenShotPath);
            }
        }
        console.log('point2');
        // set download location 
        const client = await page.target().createCDPSession();
        await client.send('Page.setDownloadBehavior', {
            behavior: 'allow',
            downloadPath: downloadPath,
            eventsEnabled: true,
        })  
        console.log(`set download path: ${downloadPath}`)
        console.log('point3');
          
        await page.goto(url, {timeout: 30000, waitUntil: 'networkidle2'});
        await page.screenshot({path: download_root + 'screen_before_click_download.png'});
        console.log('point4');
        // await page.waitForSelector('#passcode');
        try{
              await page.type('#password', password);
              await delay(randomInteger(1000, 5000)); // random delay betn 1 and 5 seconds
              
              console.log('point5');
              // submit password
              await page.click('.submit');
            } catch(err) {
              await page.type('#passcode', password);
              await delay(randomInteger(1000, 5000)); // random delay betn 1 and 5 seconds
              
              // solving capatcha
              // That's it, a single line of code to solve reCAPTCHAs 🎉
              // try{
              //   await page.solveRecaptchas()
              //   await Promise.all([
              //     page.waitForNavigation(),
              //     page.click(`#recaptcha-demo-submit`)
              //   ])
              // } catch(err){console.log(err)}
              
              // submit password
              await page.click('#passcode_btn');
              console.log(`downloading... link_index:${current_link_index}` + String(url));
              
              // update borrowed list
              await async_wait_and_update_current_download_progress(35000, borrowed )
              
               // random delay betn 35 and 80 seconds after each download click  
              await delay(randomInteger(35000, 80000));
            }
          
          // screenshot before each delay :: 10 screenshots
          await page.screenshot({path: screenShotPath + current_link_index +'.png'});
          
          // display any error by zoom <zoom sometimes give '401 unauthorized' error >
          // zoom displaying error in two ways sometimes first way, second_way other times
          let error_element = await page.$('.zm-alert__content');
          let second_error_element = await page.$('#error_msg');
          if (error_element == null && second_error_element != null)error_element = second_error_element
          
          if (error_element != null){ 
            let error_value = await page.evaluate(el => el.textContent, error_element)
            if (!(String(error_value) == '')){
              // storing error log
              append_error_logs({courseName: path, password: password, url: url, error_value: error_value}, 'link_logs');
            
              // displaying error message in console 
              console.log('Error:\t' + error_value);
              console.log('On Link: ' + link);
              // delay_sync(180000); // 3 minutes delay
              break
              // process.exit("Exit: this is the error of zoom (maybe wait few minutes and re-run the script)");
              //continue;
            }
          }

        
      } catch (error) {
        // storing error log
        append_error_logs({link:link, error_value: JSON.stringify(error)},'link_logs');

        // displaying error message in console 
        console.log('\n' + 'Error' + error + '\n');
        console.log('On Link: ' + link);
        
        console.log('Waiting 5 minutes to let pending downloads to finish');
        delay_sync(300000); // 5 minutes delay
        break
      }  
      }

      console.log('\n\n --------------- ************************* --------------- ');
      console.log(' --- completed clicking download btn -> download in progress --- \n ------------ waiting 10 minutes ------------ ');
      console.log(' --------------- ************************* --------------- \n\n ');
      delay_sync(180000);
      process.exit("Exit: this is the error of zoom (maybe wait few minutes and re-run the script)");
    // waiting 15 minutes before closing browser after clicking download to all links of specific sheet
    console.log(`\n Closing sheet: ${current_sheet} browser after waiting for 10 minutes after \"last link download click\" for download to complete. \n`);
    await browser.close();
}

let urls_path = download_root + "to_download_data_v3.json";
var links_to_download = load_json_data(urls_path);
links_to_download = links_to_download['data'];

// -------------------------------------------
// -----  Main function call to download -----
// -------------------------------------------
try{
    download_links(links_to_download, skip_error_logs=true);
} catch (error) {
  // store error reading the link: link_path, error_message
  append_error_logs({linksPath:urls_path, error_msg: 'error reading links file', error_value: String(error)}, 'other_logs');
}



/*
- auto-find & download links not to download by folder_name & file_name
- 'to_download_data_v3.json' stores url, path:<folder_name>, password
- 'progress_logs_v3.json' stores index of links not downloaded :: not used anymore
- removed solveRecaptchas() :: not working
- zoom password submit showing two varients
- zoom password input id: #passcode           previous: #password
- zoom password submit  : #passcode_btn       previous: #submit
- error message:          .zm-alert__content  previous: #error_msg
*/

Overwriting ./drive/MyDrive/zoom_downloads/download_code_v3_1.js


In [ ]:
!node ./drive/MyDrive/zoom_downloads/download_code_v3_1.js

## ***Post Processing***
* ***Delete Files with size less than 2MB***


In [43]:
%%writefile delete_below_threshold.py
import os

def delete_small_files(directory, threshold_size):
    delete_count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".mp4"):
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                if file_size < threshold_size:
                    os.remove(file_path)
                    # print(f"Deleted file: {file_path}")
                    delete_count += 1
    print(f'\n no. of files deleted that are less than {threshold_size/(1024*1024)} MB : {delete_count} ')

# Usage
directory_path = './drive/MyDrive/zoom_downloads/'
threshold_size = 10 * 1024 * 1024  # 10MB in bytes

delete_small_files(directory_path, threshold_size)

Writing delete_below_threshold.py


In [ ]:
!python3 post_process.py


 no. of files deleted that are less than 2 MB : 0 


# ***Infinite loop***

* Shell Script code
- to retry download when script gets <401:Forbidden> error from zoom



In [44]:
%%writefile ./loop_script.sh
#!/bin/bash

while true; do
    # python3 delete_below_threshold.py
    node filter_by_search.js
    python3 python_code.py
    node drive/MyDrive/zoom_downloads/download_code_v3_1.js
    sleep 3m
done

Writing ./loop_script.sh


In [ ]:
!chmod +x loop_script.sh
!/bin/bash loop_script.sh

7288
all_links_saved 1705
---------------------------------------
successfully saved all links
---------------------------------------
index 1703
todownload 745
[19, 20, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060, 1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104, 1105,

## ***Test if fuse_links path is correct***

In [36]:

%%writefile test_links.js
'use strict';
const excelToJson = require('convert-excel-to-json');
const puppeteer = require('puppeteer');
var fs = require('fs'); // to create folder if not exist // reference: https://colab.research.google.com/drive/168X6Zo0Yk2fzEJ7WDfY9Q_0UOEmHSrZc?usp=sharing
const delay = ms => new Promise(resolve => setTimeout(resolve, ms));

// for colab code
const linksPath = './drive/MyDrive/zoom_downloads/Fuse-Links.xlsx';
const download_root = './drive/MyDrive/zoom_downloads/';  // create a folder called  'zoom_downloads' in MyDrive where downloads are to be stored


let load_json_data = (file_path) => {
  // loading error links
  // var saved_data;
  try {
      var saved_data = fs.readFileSync(file_path, 'utf-8');
      saved_data = JSON.parse(saved_data);
      // console.log('Loaded Links...: \n ' + saved_data);
      return saved_data;
  } catch (error) {
      console.log('Error Loading json file ...: \n ' + file_path + error); 
  }
}
/*
    let download_progress = load_json_data(download_root + 'progress_logs.json');
    var all_sheets_links = excelToJson({  sourceFile: linksPath  });
    for (let current_sheet in all_sheets_links ){
      console.log(`{current_sheet} :: {typeof(current_sheet)}`);
      console.log(download_progress[current_sheet]);
      }
    //console.log(all_sheets_links);
*/

function exist_in_error_logs(url){
    let error_logs = load_json_data(download_root + 'error_logs.json');
    for (let error_url of error_logs['link_logs'] ){
      // console.log('linke: ',error_url);
      // console.log('\nurle:',error_url.url);
      if (error_url['url'] == url){
        return true;
      }
    }
    return false;
    // console.log(error_logs['link_logs'].length, error_logs['link_logs'])
}

let url = 'https://zoom.us/rec/download/PRQL990wJN57HsnS5z7UMJ6aR1UdPxhmJME5e8I34vcCiGOVGKakGKTALrE6OkoKtf0dbJPfGj95TOeW.NwAWgq1cnJo0M6Gi';
console.log(exist_in_error_logs(url));

Overwriting test_links.js


In [38]:
# run test_links: should display links in output 
!node test_links.js
#!ls drive/MyDrive

linke:  {
  courseName: 'Microprocessor - Section-01',
  grade: 'Year 2',
  duration: 100,
  startDateTime: '2020-11-10T14:08:00.000Z',
  endDateTime: '2020-11-10T15:48:00.000Z',
  instructorName: 'Baikuntha Kumar Acharya',
  password: 'o5o##Ce9',
  url: 'https://zoom.us/rec/download/9g7y_Va1WmQiWAKMdBQXl5aYxY761qengh5_1rpqMn_Jlp9N3s7rhsZksjZasyS2p-s_9IG36DhN7UQr.Ter-9td7dZlYwV3F',
  error_value: 'error: Http 401 error'
}

urle: https://zoom.us/rec/download/9g7y_Va1WmQiWAKMdBQXl5aYxY761qengh5_1rpqMn_Jlp9N3s7rhsZksjZasyS2p-s_9IG36DhN7UQr.Ter-9td7dZlYwV3F
linke:  {
  courseName: 'Instrumentation I - Section-01',
  grade: 'Year 2',
  duration: 50,
  startDateTime: '2020-10-02T15:34:59.999Z',
  endDateTime: '2020-10-02T16:25:00.000Z',
  instructorName: 'Mohamad Badrudhoja',
  password: 'qL$#kv47',
  url: 'https://zoom.us/rec/download/3TMfWhCR9nYmnifdqQkRxVQk8FWU7nq6B4JupezTMIJ2wCgLqFi8pB4Ine2mnwuOqz207shcOo413Lw5.hymcrABue0PrYqjf',
  error_value: 'error: Http 401 error'
}

urle: https://zo

## ***Method1: Copy files to another google drive***
  * iterate through all the folders in source dir
  * if file doesnt exist in destination: move file from source to destination 
  * if file exist in destination: move file if size in destination is less than size in source

In [ ]:
from google.colab import drive
# Mount your first Google Drive
drive.mount('/content/drive1')

# Mount second Google Drive
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser 
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "drive2"
!google-drive-ocamlfuse "/content/gdrive/drive2"



import os
import shutil


def merge(source, destination):
    for path in os.listdir(source):
        source_path = os.path.join(source, path)
        destination_path = os.path.join(destination,path)
        
        if not os.path.isdir(source_path):
            # path is file: copy from source to destination
            
            if os.path.exists(destination_path):
                print(source_path, destination_path)
                # file exist in destination
                if os.path.getsize(source_path) > os.path.getsize(destination_path):
                    # source file is larger than destination file
                    os.remove(destination_path)
                else:
                    continue    # destination file path has larger size than source file :: skip
            shutil.copy(source_path, destination_path)
        else:
            # path is folder: 
                if not os.path.exists(destination_path):
                    # destination folder does not exist
                    os.mkdir(destination_path)
                elif not os.path.isdir(destination_path):
                    # destination exists but is file not folder
                    destination_path = os.path.join(destination, path+'_folder')
                    os.mkdir(destination_path)
                merge(source_path, destination_path)

source_dir = '/content/gdrive/drive2/zoom_downloads/'
destination_dir = '/content/drive1/MyDrive/zoom_downloads/'
merge(source_dir, destination_dir)

## ***Method2: Copy files to another google drive***
  * iterate through all the folders in source dir
  * if file doesnt exist in destination: move file from source to destination 
  * if file exist in destination: move file if size in destination is less than size in source

In [ ]:
from google.colab import drive

# Mount your first Google Drive
drive.mount('/content/drive1')

# Mount second Google Drive
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser 
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "drive2"
!google-drive-ocamlfuse "/content/gdrive/drive2"




import os
import shutil


root_dir = "/content/drive/MyDrive/mero_school/"
source_dirs = ['basic electrical engineering0', 'electromagnetics0', 'electronic devices and circuits (iii_i)0']
destination_dirs = ['basic electrical engineering', 'electromagnetics', 'electronic devices and circuits (iii_i)']

source_dir = '/content/gdrive/drive2/zoom_downloads/'
destination_dir = '/content/drive1/MyDrive/zoom_downloads/'

def move_if_not_exist_and_keep_larger_file(source, destination):
    source_dir = os.path.join(root_dir, source)
    dest_dir = os.path.join(root_dir, destination)

    for file_name in os.listdir(source_dir):
        src_file_path = os.path.join(source_dir, file_name)
        dest_file_path = os.path.join(dest_dir, file_name)

        if not os.path.exists(dest_file_path):
            shutil.move(src_file_path, dest_file_path)
            print(f"Moved {file_name} to {dest_dir}")
        else:
            src_file_size = os.path.getsize(src_file_path)
            dest_file_size = os.path.getsize(dest_file_path)
            if src_file_size > dest_file_size:
                os.remove(dest_file_path)
                shutil.move(src_file_path, dest_file_path)
                print(f"Replaced {file_name} in {dest_dir} with larger file")
            else:
                os.remove(src_file_path)
                print(f"Ignored {file_name} as it already exists in {dest_dir} and is larger")

    print("Done!")

for source, destination in zip(source_dirs, destination_dirs):
    print(f"\'{str(source)}\', \'{str(destination)}\'")
    move_if_not_exist_and_keep_larger_file(source, destination)

## ***Method3: Copy files to another google drive***
* Copies From drive2 to drive1

In [ ]:
from google.colab import drive

# Mount your first Google Drive
drive.mount('/content/drive1')



# Mount second Google Drive
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser 
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "drive2"
!google-drive-ocamlfuse "/content/gdrive/drive2"





# --------------------
# Code to copy
# --------------------

# !!ls '/content/drive1/MyDrive/Colab Notebooks'
# !cp -r '/content/drive1/MyDrive/Colab Notebooks/' '/content/gdrive/drive2/colab_notebooks_another/
# rsync -avzP /media/oldhd/ /media/newhd/

# !ls /content/drive1/MyDrive/zoom_downloads/
# !ls /content/gdrive/drive2/zoom_downloads
# !ls /content/gdrive/drive2
# !ls /content/drive1/MyDrive/


!rsync -avzP -r --progress /content/gdrive/drive2/zoom_downloads/ /content/drive1/MyDrive/zoom_downloads/

# Use the -z flag to compress data during transfer. This can significantly reduce the amount of data that needs to be transferred, especially if you are transferring large files with a lot of repeated data.
# Use the --delete flag to delete files in the destination directory that have been deleted in the source directory. This can save time by avoiding the need to transfer files that are no longer needed.
# Use the -W flag to copy whole files, rather than just the differences between files. This can be faster if you are transferring a large number of small files, since it avoids the overhead of calculating and transferring the differences.
# Use the -h flag to display output in a human-readable format. This can make it easier to see which files are being transferred, and can help you identify any problems or bottlenecks in the transfer process.
# Use the -a flag to preserve file permissions, ownership, and timestamps. This can save time by avoiding the need to recalculate these values for each file.
# Use a fast network connection. rsync is designed to transfer data efficiently over a network, so a faster connection will generally result in faster transfers.
# Use a tool like rsync+ssh to encrypt data during transfer. This can add overhead to the transfer process, but can be worth it for the added security.